<a href="https://colab.research.google.com/github/leovegamedina/projects-ml/blob/main/Titanic_Kmeans_MLCourse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Projects/Unsupervised MLA/K-Means'

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np

titanic_org = pd.read_excel(root_path + '/' + 'Titanic.xlsx')

In [3]:
titanic_org.head()

,Name,Passenger Class,Sex,Age,No of Siblings or Spouses on Board,No of Parents or Children on Board,Passenger Fare,Port of Embarkation,Life Boat,Survived
0,"Allen, Miss. Elisabeth Walton",First,Female,29.0000,0,0,211.3375,Southampton,2,Yes
1,"Allison, Master. Hudson Trevor",First,Male,0.9167,1,2,151.5500,Southampton,11,Yes
2,"Allison, Miss. Helen Loraine",First,Female,2.0000,1,2,151.5500,Southampton,NaN,No
3,"Allison, Mr. Hudson Joshua Creighton",First,Male,30.0000,1,2,151.5500,Southampton,NaN,No
4,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",First,Female,25.0000,1,2,151.5500,Southampton,NaN,No
